# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
#Loading enviroment viariables
%load_ext dotenv
%dotenv 

#ignore warnings
import warnings
warnings.filterwarnings('ignore')


In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob
import pandas as pd

#Loading the 'PRICE_DATA' variable  
PRICE_DATA = os.getenv("PRICE_DATA")

#Finding the path of all parquet files in the directory PRiCE_DATA
parquet_files = glob(os.path.join(PRICE_DATA, "*/*/*.parquet"))


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
#Adding parquet files to dd dataframe
dd_px = dd.read_parquet(parquet_files)
dd_px.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.463043,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.142937,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.652859,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.219185,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.237469,2819626,Health Care,Life Sciences Tools & Services,2000


In [5]:
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
)


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [6]:
df_feat = dd_feat.compute()

In [7]:
df_feat['rolling_average'] = df_feat.groupby('ticker')['returns'].transform(lambda x: x.rolling(10).mean())

In [8]:
df_feat.head(20)

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,hi_lo_range,rolling_average
ticker,,,,,,,,,,,,,,
HUM,2005-01-03,29.860001,29.860001,28.969999,29.160000,26.036831,1029100,Health Care,Managed Health Care,2005,NaN,NaN,0.890001,NaN
HUM,2005-01-04,29.360001,29.590000,29.030001,29.100000,25.983253,1659200,Health Care,Managed Health Care,2005,29.160000,-0.002058,0.559999,NaN
HUM,2005-01-05,29.150000,29.850000,28.920000,29.700001,26.519005,1886800,Health Care,Managed Health Care,2005,29.100000,0.020619,0.930000,NaN
HUM,2005-01-06,29.750000,30.299999,29.610001,29.639999,26.465425,1638400,Health Care,Managed Health Care,2005,29.700001,-0.002020,0.689999,NaN
HUM,2005-01-07,29.389999,29.559999,29.260000,29.420000,26.268991,1194400,Health Care,Managed Health Care,2005,29.639999,-0.007422,0.299999,NaN
HUM,2005-01-10,29.580000,30.660000,29.410000,30.650000,27.367252,2274000,Health Care,Managed Health Care,2005,29.420000,0.041808,1.250000,NaN
HUM,2005-01-11,30.570000,31.340000,30.230000,31.180000,27.840485,2466100,Health Care,Managed Health Care,2005,30.650000,0.017292,1.110001,NaN
HUM,2005-01-12,31.180000,31.549999,31.000000,31.490000,28.117281,1568800,Health Care,Managed Health Care,2005,31.180000,0.009942,0.549999,NaN
HUM,2005-01-13,31.790001,32.680000,31.750000,32.419998,28.947674,2560700,Health Care,Managed Health Care,2005,31.490000,0.029533,0.930000,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

In [9]:
#It was not necessary to convert to pandas to calculate the moving average return as that operation can be completed
#similarly to how 'returns' and 'Hi_Low_Range' were calculated. See demonstration below. It is faster to run in dask as
#converting to pandas took approximately 2 and a half minutes. 

#Calculating the rolling average without converting is much faster.
dd_feat['rolling_average'] = dd_feat.groupby('ticker')['returns'].transform(lambda x: x.rolling(10).mean())

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.